In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI


In [3]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.
KEY=os.getenv("OPENAI_API_KEY")

In [5]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

d:\mcqgen1\mcqgen1\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [34]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [10]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [11]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [37]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
""" 

In [38]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [39]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [40]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [41]:
file_path=r"D:\mcqgen1\data.txt"
with open(file_path, 'r') as file:
    TEXT = file.read()

In [42]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [43]:
### get_openai_callback is used to track cost, number of tokens etc

In [44]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

d:\mcqgen1\mcqgen1\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [50]:
json.loads(response.get('review'))

{'1': {'mcq': 'What is the scientific study of life called?',
  'options': {'a': 'Physics',
   'b': 'Biology',
   'c': 'Chemistry',
   'd': 'Geology'},
  'correct': 'b'},
 '2': {'mcq': 'Which theme in biology explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Energy processing',
   'c': 'Cell structure',
   'd': 'Genetic information'},
  'correct': 'a'},
 '3': {'mcq': 'What is the primary unit of life that processes hereditary information?',
  'options': {'a': 'Atoms', 'b': 'Molecules', 'c': 'Cells', 'd': 'Tissues'},
  'correct': 'c'},
 '4': {'mcq': 'What is the process that allows organisms to move, grow, and reproduce?',
  'options': {'a': 'Photosynthesis',
   'b': 'Respiration',
   'c': 'Energy processing',
   'd': 'Digestion'},
  'correct': 'c'},
 '5': {'mcq': 'How long ago did life on Earth emerge?',
  'options': {'a': '1 million years ago',
   'b': '500 million years ago',
   'c': '3.7 billion years ago',
   'd': '10 billion years ago'},
  'corre

In [51]:
json.loads(response.get('quiz'))

{'1': {'mcq': 'What is the scientific study of life called?',
  'options': {'a': 'Chemistry',
   'b': 'Biology',
   'c': 'Physics',
   'd': 'Geology'},
  'correct': 'b'},
 '2': {'mcq': 'Which theme in biology explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Photosynthesis',
   'c': 'Mitosis',
   'd': 'Osmosis'},
  'correct': 'a'},
 '3': {'mcq': 'What is the basic unit of life that processes hereditary information?',
  'options': {'a': 'Organelle', 'b': 'Tissue', 'c': 'Cell', 'd': 'Organ'},
  'correct': 'c'},
 '4': {'mcq': 'What process allows organisms to move, grow, and reproduce?',
  'options': {'a': 'Photosynthesis',
   'b': 'Respiration',
   'c': 'Fermentation',
   'd': 'Digestion'},
  'correct': 'b'},
 '5': {'mcq': 'What emerged more than 3.7 billion years ago on Earth?',
  'options': {'a': 'Dinosaurs', 'b': 'Mammals', 'c': 'Life', 'd': 'Humans'},
  'correct': 'c'}}

In [52]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1844
Prompt Tokens:1188
Completion Tokens:656
Total Cost:0.003094


In [55]:
quiz=response.get("quiz")
quiz=json.loads(quiz)
#Converting too data frame & csv file

quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [56]:
quiz=pd.DataFrame(quiz_table_data)
quiz.to_csv("machinelearning.csv",index=False)

In [57]:
quiz

,MCQ,Choices,Correct
0,What is the scientific study of life called?,a: Chemistry | b: Biology | c: Physics | d: Ge...,b
1,Which theme in biology explains the unity and ...,a: Evolution | b: Photosynthesis | c: Mitosis ...,a
2,What is the basic unit of life that processes ...,a: Organelle | b: Tissue | c: Cell | d: Organ,c
3,"What process allows organisms to move, grow, a...",a: Photosynthesis | b: Respiration | c: Fermen...,b
4,What emerged more than 3.7 billion years ago o...,a: Dinosaurs | b: Mammals | c: Life | d: Humans,c
